template: [ENEURO PAPER](https://www.eneuro.org/content/eneuro/12/11/ENEURO.0270-25.2025.full.pdf)

In [ ]:
import mne
from mne.preprocessing.eyetracking import read_eyelink_calibration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
from pathlib import Path
from sklearn.linear_model import HuberRegressor

# 1. Functions

## 1.1 Load ASC


In [ ]:
def load_events_from_asc(file_path: str, window_ms: float = 150.0):
    """
    Load EyeLink .asc file and return (fixations_df, saccades_df, blinks_df, raw_et).

    - Adds `event_type` to each table.
    - Marks saccades near blinks within ±window_ms via `near_blink` and `blink_dist_ms`.
    - Computes saccade amplitude in degrees (visual angle) safely.
    """
    # 1) Load raw EyeLink data
    raw_et = mne.io.read_raw_eyelink(file_path, find_overlaps=False, overlap_threshold=0.1, verbose="ERROR") # tBD: comapre to overlap = true

    # 2) Read calibration
    cals = read_eyelink_calibration(
        file_path, screen_distance=0.729, screen_size=(0.552, 0.307), screen_resolution=(1920, 1080)
    )[0]
    # 3) convert units to radians of visual angle
    print("Convert units to radians of visual angle...")
    raw_et = mne.preprocessing.eyetracking.convert_units(raw_et, calibration=cals, to="radians")
    print("Done!")

    # 4) Interpolate blinks
    raw_et = mne.preprocessing.eyetracking.interpolate_blinks(raw_et, buffer=(0.05, 0.2), interpolate_gaze=False)

    # 5) DataFrames
    et_dfs = raw_et._raw_extras[0]["dfs"]

    fixations_df = et_dfs["fixations"].copy()
    fixations_df["event_type"] = "fixation"

    blinks_df = et_dfs["blinks"].copy()
    blinks_df["event_type"] = "blink"

    saccades_df = et_dfs["saccades"].copy()
    saccades_df["event_type"] = "saccade"

    # 6) Mark saccades near blinks
    saccades_df = annotate_saccades_near_blinks_in_df(saccades_df, blinks_df, window_ms=window_ms)

    # 7) Compute saccade amplitude (deg) from start/end in visual-angle coords
    saccades_df["amplitude_deg"] = saccades_df.apply(
        lambda r: compute_saccade_amplitude_from_radians(r.sacc_start_x, r.sacc_start_y,
                                                         r.sacc_end_x, r.sacc_end_y),
        axis=1
    )

    return fixations_df, saccades_df, blinks_df, raw_et

## 1.2. Load Event TSVs

since Judith created events tsv files:

`sub-XXX_ses-001_task-freeviewing_et_events.tsv`


In [ ]:
# TBD: bidsify, such that it iterates over all folders
def load_subject_tsv(
    folder_path: Path, 
    subject_id: str
) -> pd.DataFrame:
    """
    Load a TSV file for a specific subject.
    """
    filename = f"sub-{subject_id}_ses-001_task-freeviewing_et_events.tsv"
    filepath = os.path.join(folder_path, filename)
    
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"File not found: {filepath}")
    
    return pd.read_csv(filepath, sep='\t')

## 1.3. Preprocessing
### 1.3.1. Merge Fixation Candidates
if the saccades in between are below a certain threshold (e.g. amplitude 1.0 & duration 60ms). Taken frmo [Hooge et al. (2022)](https://link.springer.com/article/10.3758/s13428-021-01723-1)

In [ ]:
def merge_fixation_candidates(events, a_min=1.0, t_min: float=0.06):
    """
    Merge consecutive fixations from the same eye and filter out small saccades.
    
    Parameters:
    -----------
    events : pandas.DataFrame
        DataFrame with eye-tracking events
    a_min : float
        Minimum saccade visual angle threshold (default: 1.0)
    t_min : float
        Minimum saccade duration threshold in seconds (default: 0.06)
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with consecutive fixations merged and small saccades filtered
    """
    # Sort by eye and onset for processing
    events = events.sort_values(['eye', 'onset'])
    
    # Filter out small saccades
    events = events[
        ~(
            (events['trial_type'] == 'saccade') &
            (events['sacc_visual_angle'] < a_min) &
            (events['duration'] < t_min)
        )
    ].reset_index(drop=True)
    
    # List to store rows to keep
    rows_to_keep = []
    i = 0
    
    while i < len(events):
        current_row = events.iloc[i].copy()
        
        # Check if current and next row are both fixations from the same eye
        if (i < len(events) - 1 and 
            events.iloc[i]['trial_type'] == 'fixation' and 
            events.iloc[i + 1]['trial_type'] == 'fixation' and
            events.iloc[i]['eye'] == events.iloc[i + 1]['eye']):
            
            # Merge with next fixation(s)
            duration_sum = current_row['duration']
            j = i + 1
            
            # Keep merging as long as the next row is also a fixation from the same eye
            while (j < len(events) and 
                   events.iloc[j]['trial_type'] == 'fixation' and
                   events.iloc[j]['eye'] == current_row['eye']):
                next_row = events.iloc[j]
                # sum over all durations
                duration_sum += next_row['duration']
                
                # Weighted average for fix_avg_x, fix_avg_y, fix_avg_pupil_size
                for c in ['fix_avg_x', 'fix_avg_y', 'fix_avg_pupil_size']:
                    current_row[c] = (
                        (current_row[c] * (duration_sum - next_row['duration']) + 
                            next_row[c] * next_row['duration']) / duration_sum
                    )
                j += 1
            
            # Update duration and end_time
            current_row['duration'] = duration_sum
            current_row['end_time'] = next_row['end_time']
            rows_to_keep.append(current_row)
            i = j  # Skip all merged rows
        else:
            # Not a fixation or no consecutive fixation to merge
            rows_to_keep.append(current_row)
            i += 1
    
    # Create new dataframe from kept rows and sort by onset only
    merged_events = pd.DataFrame(rows_to_keep)
    merged_events = merged_events.sort_values(['onset']).reset_index(drop=True)
    
    return merged_events

## 1.3.2. Annotate Blinks

!!! Warning: the code is currently not refactored for the events.tsv file #TBD

In [ ]:
def annotate_saccades_near_blinks_in_df(
    saccades_df: pd.DataFrame,
    blinks_df: pd.DataFrame,
    window_ms: float
) -> pd.DataFrame:
    """
    Flag saccades as near a blink if the blink START or END falls within
    the saccade interval expanded by ±window_ms.

    Adds:
      - near_blink (bool)
    """
    saccades = saccades_df.copy()
    w = window_ms / 1000.0  # seconds

    # get blinks
    b = blinks_df[["time", "end_time"]].to_numpy(float)
    # sort by blink start
    b = b[np.argsort(b[:, 0])]

    S = saccades[["time", "end_time"]].to_numpy(float)

    near_blinks = []
    for sac_start, sac_end in S:
        win_start = sac_start - w
        win_end   = sac_end + w

        near = False
        for blink_start, blink_end in b:
            # since sorted by blink_start, if blink_start is already post window, stop
            if blink_start > win_end:
                break

            # flag if blink start OR blink end is inside the expanded saccade window
            if (win_start <= blink_start <= win_end) or (win_start <= blink_end <= win_end):
                near = True
                break

        near_blinks.append(near)

    saccades["near_blink"] = near_blinks
    return saccades


## 1.3.1 Compute Saccade Amplitude

!!! Warning: it's currently not refactored for the events.tsv file #TBD
Is it even still necessary?

In [ ]:
def compute_angle(x1, y1, z1, x2, y2, z2):

    p1 = np.array([x1, y1, z1]) # cartesian coordinates of point 1
    p2 = np.array([x2, y2, z2]) # cartesian coordinates of point 2

    # compute the angle between p1 and p2 (in radians)
    denom = (np.linalg.norm(p1) * np.linalg.norm(p2))
    if denom == 0:
        return np.nan
    alpha_rad= np.arccos(np.clip(np.dot(p1,p2)/denom,-1.,1.))

    # convert the angle from radians to degrees
    return math.degrees(alpha_rad)

def compute_saccade_amplitude(sx, sy, ex, ey):
    # convert all to cartesian
    t1 = np.radians(sx) + np.pi/2
    p1 = np.radians(sy) + np.pi/2

    t2 = np.radians(ex) + np.pi/2
    p2 = np.radians(ey) + np.pi/2

    x1 = np.cos(p1) * np.sin(t1)
    y1 = np.sin(p1) * np.sin(t1)
    z1 = np.cos(t1)

    x2 = np.cos(p2) * np.sin(t2)
    y2 = np.sin(p2) * np.sin(t2)
    z2 = np.cos(t2)

    return compute_angle(x1, y1, z1, x2, y2, z2)

def compute_saccade_amplitude_from_radians(sx, sy, ex, ey):
    """
    Compute saccade amplitude (deg) from start/end in visual-angle radians.
    """
    # Cartesian coordinates on unit sphere
    x1 = np.cos(sy) * np.sin(sx)
    y1 = np.sin(sy) * np.sin(sx)
    z1 = np.cos(sx)

    x2 = np.cos(ey) * np.sin(ex)
    y2 = np.sin(ey) * np.sin(ex)
    z2 = np.cos(ex)

    # Angle via dot product
    dot = np.clip(x1*x2 + y1*y2 + z1*z2, -1.0, 1.0)
    alpha_rad = np.arccos(dot)
    return np.degrees(alpha_rad)


## 1.4 Plot Main Sequence

!!! Warning: the code is currently not refactored for the events.tsv file #TBD

In [ ]:
def plot_main_sequence(
    saccades_df: pd.DataFrame,
    out_path: str,
    out_file_format: str,
    by_eye: str,
    title: str,
    x_label: str,
    y_label: str,
    drop_near_blinks: bool,

    drop_ms_outliers: bool = False,
    ms_outlier_mad_thresh: float = 4.3,
):
    """
    Plots main sequence (log-log). Optionally drops main-sequence outliers and logs them.
    Assumes saccades_df already contains only valid saccades and has stable columns.
    """

    s = saccades_df.copy()

    if drop_near_blinks:
        s = s[s["near_blink"] == False]

    if by_eye != "all":
        eye_map = {"left": "L", "right": "R", "both": "both"}
        s = s[s["eye"] == eye_map[by_eye]]

    base_name = f"{title.lower().replace(' ', '_')}-{by_eye}Eyes" + ("_blinkCleaned" if drop_near_blinks else "")

    if drop_ms_outliers:
        s_flagged = detect_main_sequence_outliers(s, mad_thresh=ms_outlier_mad_thresh)

        log_main_sequence_outliers(s_flagged, out_path=out_path, base_name=base_name)

        s = s_flagged[~s_flagged["ms_is_outlier"]]

    fig, ax = plt.subplots()

    if by_eye == "all":
        for eye, sub in s.groupby("eye"):
            ax.scatter(sub["sacc_visual_angle"], sub["peak_velocity"], s=10, label=str(eye))
        ax.legend(title="Eye")
    else:
        ax.scatter(s["sacc_visual_angle"], s["peak_velocity"], s=10)

    ax.set_xscale("log"); ax.set_yscale("log")
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_title(f"{title} — {by_eye}" + (" (blink-cleaned)" if drop_near_blinks else ""))
    plt.show()

    title_map = {
            "all": "All eyes",
            "left": "Left eye only",
            "right": "Right eye only",
            "both": "Binocular only",
        }

    suffix = []
    if drop_near_blinks:
        suffix.append("blink-cleaned")
    if drop_ms_outliers:
        suffix.append(f"outliers-dropped (>{ms_outlier_mad_thresh} MAD)")
    ax.set_title(f"{title} — {title_map[by_eye]}" + (f" ({', '.join(suffix)})" if suffix else ""))

    fig.tight_layout()
    plt.show()

    if out_path is not None:
        os.makedirs(out_path, exist_ok=True)
        fname = base_name + ("_msOutliersDropped" if drop_ms_outliers else "")
        fig.savefig(os.path.join(out_path, f"{fname}.{out_file_format}"), bbox_inches="tight")

In [ ]:
# CAN BE DELETED I THINK #TBD
"""
def plot_main_sequence(df: pd.DataFrame,
                       out_path: str = None,
                       out_file_format: str = "svg",
                       by_eye: str="both",
                       title: str="Main Sequence",
                       x_label: str="Saccade amplitude (deg)",
                       y_label: str="Peak velocity (deg/s)",
                       drop_near_blinks: bool=True,
                       ):

  if by_eye not in {"all", "left", "right", "both"}:
        raise ValueError("by_eye must be one of: 'all', 'left', 'right', 'both'")

  # keep only valid saccades
  s = df.query("event_type == 'saccade'").copy()
  s = s.dropna(subset=["sacc_visual_angle","peak_velocity","eye"])
  s = s[(s["amplitude_deg"] > 0) & (s["peak_velocity"] > 0)]

  if drop_near_blinks:
    s = s.query("near_blink == False").copy()

  # plausibility
  s = s[(s["sacc_visual_angle"] > 0.1) & (s["sacc_visual_angle"] <= 30)]
  s = s[(s["peak_velocity"] > 30) & (s["peak_velocity"] <= 1000)]

  fig, ax = plt.subplots()
  if by_eye == "all":
    for eye, sub in s.groupby("eye"):
      ax.scatter(sub["sacc_visual_angle"], sub["peak_velocity"], s=10, label=str(eye))
      ax.legend(title="Eye")
  elif by_eye in {"left", "right", "both"}:
    eye_mapping = {"left": "L", "right": "R", "both": "both"}
    chosen_eye = eye_mapping[by_eye]
    s = s.query(f"eye == @chosen_eye").copy()
    ax.scatter(s["sacc_visual_angle"], s["peak_velocity"], s=10)
  else:
    ax.scatter(s["sacc_visual_angle"], s["peak_velocity"], s=10)

  title_map = {
        "all": "All eyes",
        "left": "Left eye only",
        "right": "Right eye only",
        "both": "Binocular only",
    }

  ax.set_xscale("log"); ax.set_yscale("log")
  ax.set_xlabel(x_label)
  ax.set_ylabel(y_label)
  ax.set_title(f"{title} — {title_map[by_eye]}" + (" (blink-cleaned)" if drop_near_blinks else ""))

  fig = ax.figure
  fig.tight_layout()
  plt.show()
  fig.savefig(f"{out_path}/{title.lower().replace(' ', '_')}-{by_eye}Eyes-{'_blinkCleaned' if drop_near_blinks else ''}.{out_file_format}", bbox_inches="tight")
  print(f"Plot saved to file '{out_path}/{title.lower().replace(' ', '_')}-{by_eye}Eyes{'_blinkCleaned' if drop_near_blinks else ''}.{out_file_format}'")
  """

### 1.4.1. Try Out: Drop the outliers 
by treating them as residuals of the main sequence 

In [ ]:
def detect_main_sequence_outliers(
    saccades_df: pd.DataFrame,
    mad_thresh: float = 3.0,
) -> pd.DataFrame:
    """
    Robust main-sequence outlier detection in log-log space.
    Adds: ms_pred_log10v, ms_resid, ms_is_outlier
    """
    x = np.log10(saccades_df["sacc_visual_angle"].to_numpy())
    y = np.log10(saccades_df["peak_velocity"].to_numpy())

    model = HuberRegressor().fit(x.reshape(-1, 1), y)
    yhat = model.predict(x.reshape(-1, 1))
    resid = y - yhat

    med = np.median(resid)
    mad = np.median(np.abs(resid - med))
    is_outlier = np.abs(resid - med) > (mad_thresh * mad)

    out = saccades_df.copy()
    out["ms_pred_log10v"] = yhat
    out["ms_resid"] = resid
    out["ms_is_outlier"] = is_outlier
    return out

In [ ]:
def log_main_sequence_outliers(
    s_with_flags: pd.DataFrame,
    out_path: str,
    base_name: str,
) -> dict:
    """
    Writes CSV of outliers + JSON summary. Returns summary dict.
    """
    os.makedirs(out_path, exist_ok=True)

    outliers = s_with_flags[s_with_flags["ms_is_outlier"]]
    out_file = os.path.join(out_path, f"{base_name}_outliers.csv")
    outliers.to_csv(out_file, index=False)

    summary = {
        "n_total": int(len(s_with_flags)),
        "n_outliers": int(len(outliers)),
        "outlier_rate": float(len(outliers) / len(s_with_flags)),
        "saved_to": out_file,
    }

    print("\nMain sequence outlier summary")
    print("-" * 30)
    print(f"{'Total saccades':<18}: {summary['n_total']:>6}")
    print(f"{'Outliers':<18}: {summary['n_outliers']:>6}")
    print(f"{'Outlier rate':<18}: {summary['outlier_rate'] * 100:>5.2f}%")
    print(f"{'Saved to':<18}: {summary['saved_to']}")

## 1.5. Plot Fixation Duration

!!! Warning: the code is currently not refactored for the events.tsv file #TBD

In [ ]:
def plot_fixation_duration(et_combined_df: pd.DataFrame,
                           out_path: str,
                           out_file_format: str,
                           by_eye: str,          # "all", "left", "right", "both"
                           min_ms: float=60,           # drop ultra-short blinks/micro-fixations
                           max_ms: float=1000,         # drop implausibly long fixations
                           bin_w: int=20,              # bin width in ms
                           title: str="Fixation Durations",
                           x_label: str="Fixation duration (ms)",
                           y_label: str="Count"
                           ):

    if by_eye not in {"all", "left", "right", "both"}:
        raise ValueError("by_eye must be one of: 'all', 'left', 'right', 'both'")

    # 1) Fixations only, convert seconds → ms
    fix = et_combined_df.loc[et_combined_df["event_type"] == "fixation", ["duration", "eye"]].copy()
    fix = fix.dropna(subset=["duration"])
    fix["duration_ms"] = fix["duration"] * 1000.0

    # 2) Filter by eye
    if by_eye != "all":
      eye_mapping = {"left": "L", "right": "R", "both": "both"}
      chosen_eye = eye_mapping[by_eye]
      fix = fix.query(f"eye == @chosen_eye").copy()

    # 3) Filter by plausible duration range
    dur = fix["duration_ms"]
    dur = dur[(dur >= min_ms) & (dur <= max_ms)]
    if dur.empty:
        raise ValueError("No fixation durations post filtering. Check inputs or ranges.")

    fig, ax = plt.subplots()
    ax.hist(dur)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)

    title_map = {
        "all": "All eyes",
        "left": "Left eye only",
        "right": "Right eye only",
        "both": "Binocular only",
    }
    ax.set_title(f"{title} — {title_map[by_eye]}")
    plt.show()

    fig = ax.figure
    fig.tight_layout()
    fig.savefig(f"{out_path}/{title.lower().replace(' ', '_')}-{by_eye}Eyes.{out_file_format}", bbox_inches="tight")
    print(f"Plot saved to file '{out_path}/{title.lower().replace(' ', '_')}-{by_eye}Eyes.{out_file_format}'")

## 1.6. Plot Saccade amplitude
Plot C

!!! Warning: the code is currently not refactored for the events.tsv file #TBD

In [ ]:
def saccade_amplitude(
        saccades_df,
        by_eye,
        title,
        x_label,
        y_label,
        out_path,
        out_file_format
):
    # 1) Filter by eye
    if by_eye != "all":
        eye_mapping = {"left": "L", "right": "R", "both": "both"}
        chosen_eye = eye_mapping[by_eye]
        df = saccades_df.query("eye == @chosen_eye").copy()
    else:
        df = saccades_df.copy()

    # 2) Select saccade amplitudes in degrees
    # amplitudes = df["amplitude_deg"].dropna()
    all_amplitudes = df["sacc_visual_angle"].dropna()
    amplitudes = all_amplitudes[all_amplitudes <= 40]

    # Identify dropped outliers
    dropout = len(all_amplitudes[all_amplitudes > 40])

    # Report counts
    print(f"Total saccades: {len(all_amplitudes)}")
    print(f"Kept saccades (<=40°): {len(amplitudes)}")
    print(f"Dropped outliers (>40°): {dropout}, {(dropout/len(all_amplitudes))*100:.2f}%")

    # 3) Create figure
    fig, ax = plt.subplots(figsize=(5, 4))

    ax.hist(
        amplitudes,
        bins=40,
        edgecolor="black"
    )

    # 4) Labels & title
    title_map = {
        "all": "All eyes",
        "left": "Left eye only",
        "right": "Right eye only",
        "both": "Binocular only",
    }

    ax.set_title(f"{title} — {title_map[by_eye]}")
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_xlim(left=0)

    fig.tight_layout()

    # 5) Save & show
    out_file = f"{out_path}/{title.lower().replace(' ', '_')}-{by_eye}Eyes.{out_file_format}"
    fig.savefig(out_file, bbox_inches="tight")
    print(f"Plot saved to file '{out_file}'")
    plt.show()
    plt.close(fig)


## 1.7. Plot Saccade Duration
Plot D

!!! Warning: the code is currently not refactored for the events.tsv file #TBD

In [ ]:
def saccade_duration(
        saccades_df,
        by_eye,
        title,
        x_label,
        y_label,
        out_path,
        out_file_format,
        max_saccade_duration: int = 120
        ):

    # 1) Filter by eye
    if by_eye != "all":
        eye_mapping = {"left": "L", "right": "R", "both": "both"}
        chosen_eye = eye_mapping[by_eye]
        df = saccades_df.query("eye == @chosen_eye").copy()
    else:
        df = saccades_df.copy()

    # 2) Convert duration from seconds to milliseconds
    durations = (df["duration"] * 1000).dropna()

    # 3) Drop saccades >120ms
    if max_saccade_duration is not None:
        durations = durations[durations <= max_saccade_duration]
        durations_copy = durations.copy()
        dropout = len(durations_copy[durations > max_saccade_duration])
        print(f"Dropped {(dropout/len(durations))*100:.2f}% samples of duration > {max_saccade_duration} milliseconds.")

    # 4) Create figure
    fig, ax = plt.subplots(figsize=(5, 4))

    ax.hist(
        durations,
        bins=40,
        edgecolor="black"
    )

    # 5) Labels & title
    title_map = {
        "all": "All eyes",
        "left": "Left eye only",
        "right": "Right eye only",
        "both": "Binocular only",
    }

    ax.set_title(f"{title} — {title_map[by_eye]}")
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_xlim(left=0)

    fig.tight_layout()

    # 6) Save pre show
    out_file = f"{out_path}/{title.lower().replace(' ', '_')}-{by_eye}Eyes.{out_file_format}"
    fig.savefig(out_file, bbox_inches="tight")
    print(f"Plot saved to file '{out_file}'")

    plt.show()
    plt.close(fig)

## 1.8. Fixation Frequency
PLOT B

!!! Warning: the code is currently not refactored for the events.tsv file #TBD

In [ ]:
def fixation_frequency(fixations_df,
                       by_eye,
                       title,
                       x_label,
                       y_label,
                       out_path,
                       out_file_format
                       ):
    # 1) Filter by eye
    if by_eye != "all":
        eye_mapping = {"left": "L", "right": "R", "both": "both"}
        chosen_eye = eye_mapping[by_eye]
        fixations_df = fixations_df.query("eye == @chosen_eye").copy()
    else:
        fixations_df = fixations_df.copy()

    # 1) calculate in which second the fixation happened
    fixations_df["sec"] = fixations_df["time"].astype(float).floordiv(1).astype(int)

    # 2) Count fixations per second
    fix_per_sec = fixations_df.groupby("sec").size()  # pd.Series w/ index = second, value = count

    # 3) Create figure
    plt.figure()
    plt.hist(
        fix_per_sec.values,
        bins=np.arange(fix_per_sec.max() + 2) - 0.3,
        width=0.6,
    )

    plt.xlim(left=-0.3)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)

    # 6) Save pre show
    out_file = f"{out_path}/{title.lower().replace(' ', '_')}-{by_eye}Eyes.{out_file_format}"
    plt.savefig(out_file, bbox_inches="tight")
    print(f"Plot saved to file '{out_file}'")

    plt.show()

## 1.9. Saccade angular histogram
PLOT F

!!! Warning: the code is currently not refactored for the events.tsv file #TBD

In [ ]:
def saccade_angular_histogram(saccades_df,
                              by_eye,
                              title,
                              cart_title,
                              out_path,
                              out_file_format,
                              refinement = False
                              ):

    # 1) Filter by eye
    if by_eye != "all":
        eye_mapping = {"left": "L", "right": "R", "both": "both"}
        chosen_eye = eye_mapping[by_eye]
        saccades_df = saccades_df.query("eye == @chosen_eye").copy()
    else:
        saccades_df = saccades_df.copy()

    # 2) Compute saccade angles
    dx = saccades_df["sacc_end_x"] - saccades_df["sacc_start_x"]
    dy = saccades_df["sacc_end_y"] - saccades_df["sacc_start_y"]

    # Angle in radians, then degrees
    angles_rad = np.arctan2(dy, dx)
    angles_deg = np.degrees(angles_rad)

    # Optional: map to [0, 360)
    angles_deg = (angles_deg + 360) % 360

    # Convert degrees back to radians for polar plotting
    angles_rad = np.deg2rad(angles_deg)

    # 4) Refinemements (optional) -> don't seem to make a big difference
    if refinement:
        # Exclude microsaccades/artefacts
        mask = (saccades_df["amplitude_deg"] >= 1.0)
        angles_rad = angles_rad[mask]

        # Weight Saccade Amplitude
        weights = saccades_df.loc[mask, "amplitude_deg"]
        angles_rad = angles_rad[mask]
        angles_deg = angles_deg[mask]

        # 5) create Figure
        fig = plt.figure(figsize=(5, 5))
        ax = fig.add_subplot(111, polar=True)

        ax.hist(
            angles_rad,
            bins=36,        # 10° bins
            weights=weights,
            edgecolor="black"
        )
    else:
        # 5) create Figure
        fig = plt.figure(figsize=(5, 5))
        ax = fig.add_subplot(111, polar=True)

        ax.hist(
            angles_rad,
            bins=36,        # 10° bins
            edgecolor="black"
        )

    # Configure axes
    ax.set_theta_zero_location("E")   # 0° to the right
    ax.set_theta_direction(1)         # counter-clockwise
    ax.set_title(title)
    plt.tight_layout()

    # 5) Save pre show
    out_file = f"{out_path}/{title.lower().replace(' ', '_')}-{by_eye}Eyes.{out_file_format}"
    plt.savefig(out_file, bbox_inches="tight")
    print(f"Plot saved to file '{out_file}'")

    plt.show()

    # 6) Create Figure 2: Cartesian Angular Histogramm
    plt.hist(angles_deg, bins=np.arange(0, 361, 10), edgecolor="black")
    plt.xlabel("Saccade direction (deg)")
    plt.ylabel("Count")
    plt.title(cart_title)

    # 7) Save pre show
    out_file = f"{out_path}/{cart_title.lower().replace(' ', '_')}-{by_eye}Eyes.{out_file_format}"
    plt.savefig(out_file, bbox_inches="tight")
    print(f"Plot saved to file '{out_file}'")

    plt.show()


# 2. Main

!!! Warning: the most part of the code is currently not refactored for the events.tsv file #TBD

## 2.1 Load the .asc-file into a dataframe


### Specify directories and filenames

In [ ]:
asc_filepath = "C:/Users/chris/Documents/ArbeitUni/VIS_S-CCS/FreeViewing/Data/data"   # from where to load the .asc files
events_filepath = Path("C:/Users/chris/Documents/ArbeitUni/VIS_S-CCS/FreeViewing/Data/data/out")    # from where to load the .csv files, if asc was already loaded
out_dir = Path("C:/Users/chris/Documents/ArbeitUni/VIS_S-CCS/FreeViewing/Data/data/out")  # where to save the dataframes as csv files
window_size = 150.0

load all data goes through all fils and loads all files using load_events_from_asc()

In [ ]:
def load_all_data(path, window_ms: float = 150.0):
    fix_dfs = []
    sac_dfs = []
    blink_dfs = []

    for name in os.listdir(path):
        if name.endswith(".asc"):
            fixations_df, saccades_df, blinks_df, raw_et = load_events_from_asc(
                os.path.join(path, name),
                window_ms=window_ms
            )
            pid = int(name[4:7])

            fixations_df = fixations_df.copy()
            saccades_df = saccades_df.copy()
            blinks_df = blinks_df.copy()

            fixations_df["ID"] = pid
            saccades_df["ID"] = pid
            blinks_df["ID"] = pid

            fix_dfs.append(fixations_df)
            sac_dfs.append(saccades_df)
            blink_dfs.append(blinks_df)

    # big DataFrames for each event type
    all_fixations_df = pd.concat(fix_dfs, ignore_index=True)
    all_saccades_df = pd.concat(sac_dfs, ignore_index=True)
    all_blinks_df = pd.concat(blink_dfs, ignore_index=True)

    return all_fixations_df, all_saccades_df, all_blinks_df

Save the data as csv, such that I do not need to reload the data again and again

In [ ]:
def save_data_2csv(df, out_dir):

    out_dir.parent.mkdir(parents=True, exist_ok=True)
    # Save DataFrames
    df.to_csv(out_dir, index=False)

bring it all together:

In [ ]:
fixations_path = events_filepath / "fixations.csv"
saccades_path = events_filepath / "saccades.csv"
blinks_path = events_filepath / "blinks.csv"

if not (fixations_path.exists()
        and saccades_path.exists()
        and blinks_path.exists()):
    fixations_df, saccades_df, blinks_df = load_all_data(asc_filepath)

    save_data_2csv(fixations_df, fixations_path)
    save_data_2csv(saccades_df, saccades_path)
    save_data_2csv(blinks_df, blinks_path)

else:
    fixations_df = pd.read_csv(fixations_path)
    saccades_df = pd.read_csv(saccades_path)
    blinks_df = pd.read_csv(blinks_path)

In [ ]:
fixations_df

In [ ]:
saccades_df

In [ ]:
saccades_df['sacc_visual_angle'].describe()

In [ ]:
blinks_df

## 2.2. Load the Events TSV

In [ ]:
subject_id="005"
events = load_subject_tsv(events_filepath, subject_id)
events

### 2.2.1. Merge Fixation Candidates
if the saccades in between are below a certain threshold (e.g. amplitude 1.0 & duration 60ms). Taken frmo [Hooge et al. (2022)](https://link.springer.com/article/10.3758/s13428-021-01723-1)  

In [ ]:
merged_events = merge_fixation_candidates(events)

# Save to file
output_filename = f'sub-{subject_id}_ses-001_task-freeviewing_et_events_merged.tsv'
output_path = os.path.join(out_dir, output_filename)
merged_events.to_csv(output_path, sep='\t', index=False)

merged_events

### 2.2.2. Comparative Plots Pre-Post Merger

In [ ]:
def plot_eye_trace_both_eyes(events_before, events_after, time_window=None):
    """
    Plot horizontal eye position trace for both eyes with fixations overlaid.
    
    Parameters:
    -----------
    events_before : pandas.DataFrame
        Original events dataframe
    events_after : pandas.DataFrame
        Merged events dataframe
    time_window : tuple, optional
        (start_time, end_time) to zoom into specific time range
    """
    
    fig, axes = plt.subplots(2, 1, figsize=(15, 8), sharex=True)
    
    for ax_idx, eye in enumerate(['L', 'R']):
        ax = axes[ax_idx]
        
        # Filter by eye
        before = events_before[events_before['eye'] == eye].copy()
        after = events_after[events_after['eye'] == eye].copy()
        
        # Apply time window if specified
        if time_window:
            before = before[(before['onset'] >= time_window[0]) & 
                           (before['onset'] <= time_window[1])]
            after = after[(after['onset'] >= time_window[0]) & 
                         (after['onset'] <= time_window[1])]
        
        pos_col = 'fix_avg_x'
        
        before_fix = before[before['trial_type'] == 'fixation']
        after_fix = after[after['trial_type'] == 'fixation']
        
        # Create continuous trace
        times = []
        positions = []
        
        for idx, row in before_fix.iterrows():
            times.extend([row['onset'], row['end_time']])
            positions.extend([row[pos_col], row[pos_col]])
            
            next_idx = before_fix.index.get_loc(idx)
            if next_idx < len(before_fix) - 1:
                next_row = before_fix.iloc[next_idx + 1]
                times.append(next_row['onset'])
                positions.append(next_row[pos_col])
        
        # Plot the trace
        ax.plot(times, positions, 'k-', linewidth=1.5, alpha=0.8, 
                label='Eye trace' if ax_idx == 0 else '')
        
        # Add fixation boxes BEFORE merging (light blue, transparent)
        for idx, row in before_fix.iterrows():
            y_center = row[pos_col]
            box_height = 20
            
            ax.add_patch(plt.Rectangle(
                (row['onset'], y_center - box_height/2),
                row['duration'],
                box_height,
                facecolor='limegreen',
                edgecolor='mediumorchid',
                alpha=0.4,
                linewidth=1.5,
                label='Before merging' if (ax_idx == 0 and idx == before_fix.index[0]) else ''
            ))
        
        # Add fixation boxes AFTER merging (red, more opaque)
        for idx, row in after_fix.iterrows():
            y_center = row[pos_col]
            box_height = 20
            
            ax.add_patch(plt.Rectangle(
                (row['onset'], y_center - box_height/2),
                row['duration'],
                box_height,
                facecolor='tomato',
                edgecolor='crimson',
                alpha=0.6,
                linewidth=2,
                label='After merging' if (ax_idx == 0 and idx == after_fix.index[0]) else ''
            ))
        
        # Styling
        ax.set_ylabel(f'Eye {eye}\nHorizontal position', fontsize=12, fontweight='bold')
        ax.set_title(f'Eye {eye}: n={len(before_fix)} → {len(after_fix)} fixations', 
                    fontsize=11, loc='right', style='italic')
        ax.grid(True, alpha=0.3)
        
        # Add legend only to first plot
        if ax_idx == 0:
            ax.legend(loc='upper right', fontsize=10)
    
    axes[-1].set_xlabel('Time (s)', fontsize=12)
    
    fig.suptitle('Horizontal Eye Position: Fixation Merging Comparison', 
                fontsize=14, fontweight='bold', y=0.995)
    
    plt.tight_layout()
    return fig

In [ ]:
events_before = events.copy()

# With time window
fig1 = plot_eye_trace_both_eyes(events_before, merged_events, time_window=(2500, 2520)) # change the time window to inspect other parts
plt.show()

# Full timeline
fig2 = plot_eye_trace_both_eyes(events_before, merged_events)
plt.show()

# Save
fig1.savefig(os.path.join(out_dir, 'both_eyes_fixation_comparison.png'), dpi=300, bbox_inches='tight')
fig1.savefig(os.path.join(out_dir, 'both_eyes_fixation_comparison_alltimes.png'), dpi=300, bbox_inches='tight')

## 2.2 Plot the Data

### Specify general parameters

In [ ]:
by_eye = "left" # "all", "both", "left", "right"
out_path = "data/out"
out_file_format = "svg" # "pdf", "eps"

### 2.2.1 Main Sequence
specify the parameters for the main sequence plot

In [ ]:
title = "Main Sequence"
x_label = "Saccade amplitude (deg)"
y_label = "Peak velocity (deg/s)"
drop_near_blinks = True

In [ ]:
saccades_df

In [ ]:
plot_main_sequence(saccades_df,
                   out_path = out_path,
                   out_file_format = out_file_format,
                   by_eye = by_eye,
                   title = title,
                   x_label = x_label,
                   y_label = y_label,
                   drop_near_blinks = drop_near_blinks,
                   )

In [ ]:
# drop outliers as well
plot_main_sequence(
    saccades_df,
    out_path= out_path + "/try_out",
    out_file_format = out_file_format,
    by_eye=by_eye,
    title = title + " + (outlier cleaned)",
    x_label = x_label,
    y_label = y_label,
    drop_near_blinks=drop_near_blinks,
    drop_ms_outliers=True
)

#### 2.2.1.1. Try out: Main Sequence with different window sizes to drop blinks

In [ ]:
window_ms_list = [10, 30, 50, 80, 100, 120, 150, 180]
prev = 0
x_label = "Saccade amplitude (deg)"

for window_ms in window_ms_list:
    print(f"Saccades for near blinks annotated for window size {window_ms}")

    sc_df = annotate_saccades_near_blinks_in_df(
        saccades_df,
        blinks_df,
        window_ms=window_ms
    )

    plot_main_sequence(
        sc_df[sc_df["near_blink"]].copy(),
        out_path=out_path + f"/ws{window_ms}",
        out_file_format=out_file_format,
        by_eye=by_eye,
        title=title + f" ws{window_ms}REMOVED",
        x_label=x_label,
        y_label=y_label,
        drop_near_blinks=False,
    )

    plot_main_sequence(
        sc_df,
        out_path=out_path + f"/ws{window_ms}",
        out_file_format=out_file_format,
        by_eye=by_eye,
        title=title + f" ws{window_ms}",
        x_label=x_label,
        y_label=y_label,
        drop_near_blinks=drop_near_blinks,
    )

    # Sanity check
    cur = int(sc_df["near_blink"].sum())
    print(window_ms, cur, "delta", cur - prev)
    prev = cur


In [ ]:
print("drop_near_blinks =", drop_near_blinks)

In [ ]:
# Unprocessed / raw main sequence
plot_main_sequence(
    saccades_df,
    out_path=out_path + "/raw",
    out_file_format=out_file_format,
    by_eye=by_eye,
    title=title + " raw",
    x_label=x_label,
    y_label=y_label,
    drop_near_blinks=False,   # raw has no near-blink removal
)


### 2.2.2 Fixation Duration
specify the parameters for the fixation duration plot

In [ ]:
min_ms = 60,           # drop ultra-short blinks/micro-fixations
max_ms = 1000,         # drop implausibly long fixations
bin_w = 20,              # bin width in ms
title = "Fixation Durations"
x_label = "Fixation duration (ms)"
y_label = "Count"

In [ ]:
plot_fixation_duration(fixations_df,
                       by_eye = by_eye,
                       min_ms = min_ms,
                       max_ms = max_ms,
                       bin_w = bin_w,
                       title = title,
                       x_label = x_label,
                       y_label = y_label,
                       out_path = out_path,
                       out_file_format = out_file_format
                       )

### 2.2.3 Saccade Amplitude
specify the parameters for the saccade smplitude plot

In [ ]:
title = "Saccade Amplitude"
x_label = "Saccade amplitude (deg)"
y_label = "Count"

In [ ]:
saccade_amplitude(saccades_df,
                  by_eye = by_eye,
                  title = title,
                  x_label = x_label,
                  y_label = y_label,
                  out_path = out_path,
                  out_file_format = out_file_format
                  )

### 2.2.4 Saccade Duration
specify the parameters for the saccade duration plot

In [ ]:
title = "Saccade Duration"
x_label = "Saccade duration (ms)"
y_label = "Count"
max_saccade_duration = 120 # 100, 150, 200, 250, 300, none

In [ ]:
saccade_duration(saccades_df,
                  by_eye = by_eye,
                  title = title,
                  x_label = x_label,
                  y_label = y_label,
                  out_path = out_path,
                  out_file_format = out_file_format,
                  max_saccade_duration=max_saccade_duration
                  )

### 2.2.5 Fixation Frequency
specify the parameters for the fixation frequency plot

In [ ]:
x_label = "Fixation Frequency" # Maybe 'Fixations per second' would fit better?
y_label = "Count"
title = "Fixation frequency histogram"

In [ ]:
fixation_frequency(fixations_df,
                   by_eye = by_eye,
                   title = title,
                   x_label = x_label,
                   y_label = y_label,
                   out_path = out_path,
                   out_file_format = out_file_format
                   )

### 2.2.5 Saccade Direction Histogramm
specify the parameters for the fixation frequency plot

In [ ]:
title = "Saccade Direction Histogram"
cart_title = "Cartesian Angular Histogram"
refinement = False  # refinements: weight by saccade amplitude, excluding microsaccades or artifacts

In [ ]:
saccade_angular_histogram(saccades_df,
                          by_eye=by_eye,
                          title=title,
                          cart_title=cart_title,
                          out_path=out_path,
                          out_file_format=out_file_format,
                          refinement=refinement
                          )

# 3. Outlier Analysis

concat & load dfs

In [ ]:
et_all = pd.concat(
    [fixations_df, saccades_df, blinks_df],
    axis=0,
    ignore_index=True,
    sort=False
)

outliers = pd.read_csv(
    r"data/out/try_out/main_sequence_+_(outlier_cleaned)-bothEyes_blinkCleaned_outliers.csv"
) # tBD: better filenaming

create 'flags' col

In [ ]:
outlier_key_cols = [
    "ID",
    "eye",
    "time",
    "end_time"
]

outlier_flags = (
    outliers
    .loc[outliers["ms_is_outlier"]]
    [outlier_key_cols]
    .assign(is_outlier=True)
)

merge

In [ ]:
et_all = et_all.merge(
    outlier_flags,
    on=outlier_key_cols,
    how="left"
)

et_all["is_outlier"] = et_all["is_outlier"].fillna(False)

In [ ]:
et_all

Weird Tryout

In [ ]:
raw = mne.io.read_raw_eyelink(fname="data\sub-030_ses-001_et.asc")